<a href="https://colab.research.google.com/github/dmitriygorlov/Yandex.Practikum_Data_Science/blob/main/Module-02_02-Supervised-learning/project-6_supervised-lerning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Отток клиентов

Нужно постройте модель, которая спрогнозирует "уйдёт клиент из банка в ближайшее время или нет" с предельно большим значением *F1*-меры.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

## Подготовка данных

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

from sklearn.utils import shuffle


Подгрузили нужные библиотеки

In [2]:
df=pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')
display(df.head())
df.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Подгрузили файл, вывели базовую информацию

Признаки
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата
Целевой признак
- Exited — факт ухода клиента



В файле 10000 строчек, нужны следующие действия по подготовке:
- убрать признаки явно не влияющие на расчёт (не имеющие смысла, даже если зависимость есть) Номер строки, Номер клиента, Фамилия
- в столбцу Tenure (сколько лет человек является клиентом банка) есть пропуски, нужно изучить и что-нибудь сделать с ними
- превратить в численные значения Страну и Пол с помощью One-hot-encoding
- Масштабировать численные признаки Баланса и Зарплаты
- Поделить выборки на обучающую, валидационную и тестовую (3:1:1)

In [3]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
display(df.head())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
print(df['Exited'].value_counts(normalize = True))
print(df[df['Tenure'].isna()]['Exited'].value_counts(normalize = True))
print()
print('Процент пустых значений равен {:.1%}'.format(df['Tenure'].isna().sum()/len(df)))
print()
display(df[df['Tenure'].isna()].head(10))

0    0.7963
1    0.2037
Name: Exited, dtype: float64
0    0.79868
1    0.20132
Name: Exited, dtype: float64

Процент пустых значений равен 9.1%



,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
82,543,France,Female,36,NaN,0.00,2,0,0,26019.59,0
85,652,Spain,Female,75,NaN,0.00,2,1,1,114675.75,0
94,730,Spain,Male,42,NaN,0.00,2,0,1,85982.47,0
99,413,France,Male,34,NaN,0.00,2,0,0,6534.18,0
111,538,Germany,Male,39,NaN,108055.10,2,1,0,27231.26,0


Пропуски равноценно распределены, закономерностей нет, 9% от общих данных. Исходя из незначительности - выкинем их

In [5]:
print(len(df))
df.dropna(inplace = True)
print(len(df))

10000
9091


In [6]:
df_OHE = pd.get_dummies(df, drop_first = True)

display(df_OHE.head())

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [7]:
scaler = StandardScaler()
df_OHE_scal = df_OHE
df_OHE_scal[['EstimatedSalary', 'Balance']] = scaler.fit_transform(df_OHE[['EstimatedSalary', 'Balance']])
display(df_OHE_scal.head())

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,-1.227780,1,1,1,0.020264,1,0,0,0
1,608,41,1.0,0.116887,1,0,1,0.214527,0,0,1,0
2,502,42,8.0,1.333920,3,1,0,0.238632,1,0,0,0
3,699,39,1.0,-1.227780,2,0,0,-0.110281,0,0,0,0
4,850,43,2.0,0.785996,1,1,1,-0.366132,0,0,1,0


<div class="alert alert-block alert-info">
<b>Совет: </b> Scaler желательно обучать только после разбиения выборки на части. При этом он должен быть обучен только на тренировочной части данных. Это позволит уменьшит переобучение.
</div>

In [8]:
features = df_OHE_scal.drop(['Exited'], axis = 1)
target = df_OHE_scal['Exited']

features_main, features_test, target_main, target_test = train_test_split(features, target, 
                                                                                         test_size = 0.2, random_state = 12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_main, target_main, 
                                                                                         test_size = 0.25, random_state = 12345)
print('Тренировочных объектов {:.0%}'.format(len(target_train)/len(target)))
print('Валидационных объектов {:.0%}'.format(len(target_valid)/len(target)))
print('Тестовых объектов {:.0%}'.format(len(target_test)/len(target)))

Тренировочных объектов 60%
Валидационных объектов 20%
Тестовых объектов 20%


Все подготовительные работы сделаны, можно приступать к изучению и обучению моделей. Наша задача - это задача бинарной классификации

## Исследование задачи

Исследуем баланс классов

In [9]:
print(target_train.value_counts(normalize = True))

0    0.794829
1    0.205171
Name: Exited, dtype: float64


Среди классов есть дисбаланс: 20% объектов положительные, остальные отрицательные. Построим модель регрессии и случаного леса на дисбалансированных данных

# Построим модели

In [10]:
# Модель регрессии

model_lr = LogisticRegression(solver='liblinear', random_state=12345)
model_lr.fit(features_train, target_train)
predicted_valid = model_lr.predict(features_valid)
print('F1 score = {:.2f}'.format(f1_score(target_valid, predicted_valid)))

F1 score = 0.30


Качество модели Регрессии весьма низкое. F1 score = 0,3. Попробуем модель случайного леса

In [11]:
# Модель случайного леса
forest_n_est = 0
forest_max_dep = 0
forest_f1 = 0
forest_acc = 0


for est in range (10, 101, 10):
    for dep in range (5, 30, 1):
        model_rf = RandomForestClassifier(n_estimators = est, max_depth = dep, random_state=12345)
        model_rf.fit(features_train, target_train)
        predicted_valid = model_rf.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        acc = accuracy_score(target_valid, predicted_valid)
        if f1 > forest_f1:
            forest_f1 = f1
            forest_max_dep = dep
            forest_n_est = est
            forest_acc = acc

print('N_estim (количество деревьев) = {:.0f}'.format(forest_n_est))
print('Max_depth (максимальная глубина) = {:.0f}'.format(forest_max_dep))
print('F1 score = {:.2f}'.format(forest_f1))

N_estim (количество деревьев) = 40
Max_depth (максимальная глубина) = 20
F1 score = 0.56


С моделью случайного леса модель гораздо лучше. Перебрав гипермараметры, мы нашли лучшую модель при количестве еревьев 40 и максимальной глубине 20. Однако качество модели не дотягивает до необходимых 0,59. F1 score = 0,56 (лучше, чем в предыдущей модели)

**Краткий вывод** При дисбалансе классов (положительные исходы только 20%) мы не можем добиться хороших результатов по метрике F1 (требуемая 0,59). Максимально хороший параметр мы получили на модели случайного леса с количеством деревьев 40 и максимальной глубиной 20 (F1 = 0,56) Поэтому необходимо сбалансировать классы и подобрать лучшую модель перед окончательным тестом.

## Борьба с дисбалансом

Улучшим качество модели, применим вначале upsampling и применим модель линейной регрессии и найдём лучшую модель случайного леса, а затем downsampling и также найдем лучшую модель.

In [12]:
def upsampling (features, target, repeat):
    features_zero = features[target == 0]
    features_one = features[target == 1]
    target_zero = target[target == 0]
    target_one = target[target == 1]
    
    features_final = pd.concat([features_zero] + [features_one] * repeat)
    target_final = pd.concat([target_zero] + [target_one] * repeat)
    
    features_final, target_final = shuffle(
        features_final, target_final, random_state=12345)
    
    return features_final, target_final

features_up_train, target_up_train = upsampling(features_train, target_train, 4)

Провели upsampling (умножили на 4, чтобы из изначального соотношения 80:20, стало 80:80), теперь применим на моделях регрессии и случайного леса

In [13]:
# Модель регрессии

model_lr = LogisticRegression(solver='liblinear', random_state=12345)
model_lr.fit(features_up_train, target_up_train)
predicted_valid = model_lr.predict(features_valid)
print('F1 score = {:.2f}'.format(f1_score(target_valid, predicted_valid)))

probabilities_valid = model_lr.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print('AUC-ROC = {:.2f}'.format(roc_auc_score(target_valid, probabilities_one_valid)))

F1 score = 0.50
AUC-ROC = 0.78


Качество модели со сбалансированными классами при upsampling стало куда лучше (F1 стал 0,5 против 0,3 на несбалансированных),  весьма низкое. AUC-ROC = 0,78. Попробуем модель случайного леса

In [14]:
# Модель случайного леса
forest_n_est = 0
forest_max_dep = 0
forest_f1 = 0
forest_acc = 0


for est in range (10, 101, 10):
    for dep in range (5, 30, 1):
        model_rf = RandomForestClassifier(n_estimators = est, max_depth = dep, random_state=12345)
        model_rf.fit(features_up_train, target_up_train)
        predicted_valid = model_rf.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > forest_f1:
            forest_f1 = f1
            forest_max_dep = dep
            forest_n_est = est

print('N_estim (количество деревьев) = {:.0f}'.format(forest_n_est))
print('Max_depth (максимальная глубина) = {:.0f}'.format(forest_max_dep))
print('F1 score = {:.2f}'.format(forest_f1))

probabilities_valid = model_rf.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print('AUC-ROC = {:.2f}'.format(roc_auc_score(target_valid, probabilities_one_valid)))

N_estim (количество деревьев) = 40
Max_depth (максимальная глубина) = 9
F1 score = 0.61
AUC-ROC = 0.83


Модель случайного леса с upsampling показала себя лучше и превзошла необходимых порог значения F1 меры в 0,59 (у модели с 40 деревьями и 9 максимальной глубины F1 = 0,61). AUC-ROC при нашей модели равен 0,83, что тоже лучше, чем в линейной регрессии. Теперь пройдёмся мтодом downsampling

In [15]:
def downsampling (features, target, fraction):
    features_zero = features[target == 0]
    features_one = features[target == 1]
    target_zero = target[target == 0]
    target_one = target[target == 1]
    
    features_final = pd.concat([features_zero.sample(frac=fraction, random_state=12345)] + [features_one])
    target_final = pd.concat([target_zero.sample(frac=fraction, random_state=12345)] + [target_one])
    
    features_final, target_final = shuffle(
        features_final, target_final, random_state=12345)
    
    return features_final, target_final

features_down_train, target_down_train = downsampling(features_train, target_train, 0.25)

Провели downsampling (взяли 1/4, чтобы из изначального соотношения 80:20, стало 20:20), теперь применим на моделях регрессии и случайного леса

In [16]:
# Модель регрессии

model_lr = LogisticRegression(solver='liblinear', random_state=12345)
model_lr.fit(features_down_train, target_down_train)
predicted_valid = model_lr.predict(features_valid)
print('F1 score = {:.2f}'.format(f1_score(target_valid, predicted_valid)))

probabilities_valid = model_lr.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print('AUC-ROC = {:.2f}'.format(roc_auc_score(target_valid, probabilities_one_valid)))

F1 score = 0.50
AUC-ROC = 0.78


Качество модели со сбалансированными классами при downsampling такое же как при upsampling (F1 равен 0,5, AUC-ROC равен 0,78). Попробуем модель случайного леса

In [17]:
# Модель случайного леса
forest_n_est = 0
forest_max_dep = 0
forest_f1 = 0
forest_acc = 0


for est in range (10, 101, 10):
    for dep in range (5, 30, 1):
        model_rf = RandomForestClassifier(n_estimators = est, max_depth = dep, random_state=12345)
        model_rf.fit(features_down_train, target_down_train)
        predicted_valid = model_rf.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > forest_f1:
            forest_f1 = f1
            forest_max_dep = dep
            forest_n_est = est

print('N_estim (количество деревьев) = {:.0f}'.format(forest_n_est))
print('Max_depth (максимальная глубина) = {:.0f}'.format(forest_max_dep))
print('F1 score = {:.2f}'.format(forest_f1))

probabilities_valid = model_rf.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print('AUC-ROC = {:.2f}'.format(roc_auc_score(target_valid, probabilities_one_valid)))

N_estim (количество деревьев) = 80
Max_depth (максимальная глубина) = 9
F1 score = 0.59
AUC-ROC = 0.84


Модель случайного леса с downsampling показала себя хуже, чем при upsampling (F1 мера равна 0,59 против 0,61). AUC-ROC примерно такой же (0,84 против 0,83 предыдущей)

**Краткий вывод** Сбалансированная модель показывает лучшие результаты, наилучшей стала модель случайного леса (60 деревьев, глубина 13) после upsampling. Значение F1 меры побило порог и равно 0,61. AUC-ROC модели равен 0,83. Перейдём к тестированию лучшей модели

## Тестирование модели

Проведите финальное тестирование.

In [19]:
best_model = RandomForestClassifier(n_estimators = 40, max_depth = 9, random_state=12345)
best_model.fit(features_up_train, target_up_train)

predicted_valid = best_model.predict(features_test)
f1 = f1_score(target_test, predicted_valid)

print('F1 score = {:.2f}'.format(forest_f1))

probabilities_test = best_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

print('AUC-ROC = {:.2f}'.format(roc_auc_score(target_test, probabilities_one_test)))

F1 score = 0.59
AUC-ROC = 0.86


На тестовой выборке модель показала себя достаточно хорошо, выполнив порог F1 меры в 0,59. При этом значение AUC-ROC тоже высокое 0,85

**Общий вывод:** Теперь мы можем пресказывать уход клиента из банка по модели случайного леса (количество деревьев 40, максимальная глубина 9), обучив её на сбалансированной выборке методом upsampling. Модель была успешна протестирована на Тестовых данных
- Важно помнить, что дисбаланс классов очень велик, поэтому следуюет уделять этому внимание при дальнейшем использовании модели!